In [1]:
# This file is used to test how to get data from the Frax API and supplied info.
# We may also sanity check or use chain access via a local node or blockchain scanner
# if that's necessary

In [2]:
from cache import cache
from collections import defaultdict
from decimal import Decimal
import httpx
import json
from pprint import pprint
import sys
from tabulate import tabulate
from web3 import Web3

In [4]:
# Get Gauge Pools and sort by gauge_index
req = cache.get('https://api.frax.finance/gauge/info')
gauges = json.loads(req['value'])
'''
# A gauge looks like:

{'address': '0x3EF26504dbc8Dd7B7aa3E97Bc9f3813a9FC0B4B0',
 'fxs_next': 39981.83490806742,
 'fxs_now': 33694.5538302253,
 'gauge_index': 0,
 'gauge_weight': '17127149135922894237322600',
 'name': 'Uniswap V3 FRAX/USDC',
 'relative_weight_next_pct': 45.69352560921991,
 'relative_weight_next_raw': '456935256092199104',
 'relative_weight_now_pct': 38.508061520257485,
 'relative_weight_now_raw': '385080615202574845'}
'''
gauges.sort(key=lambda g: g['gauge_index'])
for gauge in gauges:
  print(gauge['name'])
  
# We can also get some of this information from easily from the blockchain
# Note for each chain you'd need to query a node (and potentially a blockchain scanner for ABI)

# We use etherscan for getting the ABI and contract details easier...
ETHERSCAN_KEY = "YourApiKeyToken"

def get_abi(contract):
  URL = f'https://api.etherscan.io/api?module=contract&action=getabi&address={contract}&apikey={ETHERSCAN_KEY}'
  req = cache.get(URL)
  r = json.loads(req['value'])
  return r['result']

def get_contract_details(contract):
  URL = f'https://api.etherscan.io/api?module=contract&action=getsourcecode&address={contract}&apikey={ETHERSCAN_KEY}'
  req = cache.get(URL)
  r = json.loads(req['value'])
  return r['result'][0]

# Use a local node...
w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))

GAUGE = '0x3669c421b77340b2979d1a00a792cc2ee0fce737'
GAUGE = Web3.toChecksumAddress(GAUGE)
c_gauge = w3.eth.contract(address=GAUGE, abi=get_abi(GAUGE))

farms = []
n = c_gauge.functions.n_gauges().call()
for i in range(n):
  r = c_gauge.functions.gauges(i).call()
  farms.append(r)

# Get farm details
print()
for farm in farms:
  details = get_contract_details(farm)
  print(details['ContractName'])
  print(farm)

  c_farm = w3.eth.contract(address=farm, abi=details['ABI'])

  try:
    reward_tokens = c_farm.functions.getAllRewardTokens().call()
    print('Reward Tokens:')
    pprint(reward_tokens)
  except:
    print('No getAllRewardTokens() - FXS? 0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0')
    pass
  print()

Uniswap V3 FRAX/USDC
mStable FRAX/mUSD
Uniswap V3 FRAX/DAI
Sushi FRAX/SUSHI
StakeDAO sdFRAX3CRV-f
Gelato Uniswap FRAX/DAI
StakeDAO sdETH-FraxPut
Uniswap V3 FRAX/agEUR
Vesper Orbit FRAX
Temple FRAX/TEMPLE
Curve VSTFRAX-f

FraxFarm_UniV3_veFXS_FRAX_USDC
0x3EF26504dbc8Dd7B7aa3E97Bc9f3813a9FC0B4B0
No getAllRewardTokens() - FXS? 0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0

FraxMiddlemanGauge_FRAX_mUSD
0x3e14f6EEDCC5Bc1d0Fc7B20B45eAE7B1F74a6AeC
No getAllRewardTokens() - FXS? 0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0

FraxUniV3Farm_Stable_FRAX_DAI
0xF22471AC2156B489CC4a59092c56713F813ff53e
No getAllRewardTokens() - FXS? 0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0

StakingRewardsMultiGauge_FRAX_SUSHI
0xb4Ab0dE6581FBD3A02cF8f9f265138691c3A7d5D
Reward Tokens:
['0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0',
 '0x6B3595068778DD592e39A122f4f5a5cF09C90fE2']

StakingRewardsMultiGauge_StakeDAO
0xEB81b86248d3C2b618CcB071ADB122109DA96Da2
Reward Tokens:
['0x3432B6A60D23Ca0dFCa7761B7ab56459D9C964D0',
 '0x739

In [5]:
# Get all pools listed in the "Staking" tab
req = cache.get('https://api.frax.finance/pools')
pools = json.loads(req['value'])
'''
# A pool looks like
{'apy': 18.786202278137203,
  'apy_max': 56.35860683441161,
  'chain': 'arbitrum',
  'identifier': 'Curve VSTFRAX-f',
  'is_deprecated': False,
  'liquidity_locked': 24336315.042461667,
  'logo': '',
  'pair': 'Curve VSTFRAX-f',
  'pairLink': 'https://app.frax.finance/staking#Curve_VSTFRAX_F',
  'platform': 'curve_arbi_vstfrax',
  'pool_rewards': ['FXS', 'VSTA'],
  'pool_tokens': ['FRAX', 'VST']},
'''

# Huge PITA but we can get contract addresses for all these pools as well...
req = cache.get('https://raw.githubusercontent.com/FraxFinance/frax-solidity/master/src/types/constants.ts')
constants = req['value'].decode('utf8')
start = constants.find('export const CONTRACT_ADDRESSES')
end = constants.find('export const INVESTOR_ALLOCATIONS')
constants = constants[start:end].replace('export const CONTRACT_ADDRESSES = ', '')

# best tool: https://regex101.com/
# also useful: https://pythex.org/
# docs: https://note.nkmk.me/en/python-str-replace-translate-re-sub/
# https://jsonlint.com/
import re
constants = re.sub(r'//.*', '', constants)
constants = re.sub(r'[\s]*NOTE.*', '', constants)
constants = re.sub(r'([a-zA-Z0-9_]+)[\s]*:\B', r'"\1":', constants)
import ast
contracts = ast.literal_eval(constants)

staking_contracts = {}
for key in contracts:
  if 'staking_contracts' in contracts[key]:
    for contract in contracts[key]['staking_contracts']:
      staking_contracts[contract] = {'chain': key, 'address': contracts[key]['staking_contracts'][contract]}
    
data = []
for gauge in gauges:
  try:
    contract = staking_contracts[gauge['name']]
    data.append((contract['chain'], gauge['name'], contract['address']))
  except:
    print(f'NO CONTRACT for {gauge["name"]}')

print(tabulate(data, headers=['chain', 'gauge', 'address']))


chain     gauge                    address
--------  -----------------------  ------------------------------------------
ethereum  Uniswap V3 FRAX/USDC     0x3EF26504dbc8Dd7B7aa3E97Bc9f3813a9FC0B4B0
polygon   mStable FRAX/mUSD        0xc425Fd9Ed3C892d849C9E1a971516da1C1B29696
ethereum  Uniswap V3 FRAX/DAI      0xF22471AC2156B489CC4a59092c56713F813ff53e
ethereum  Sushi FRAX/SUSHI         0xb4Ab0dE6581FBD3A02cF8f9f265138691c3A7d5D
ethereum  StakeDAO sdFRAX3CRV-f    0xEB81b86248d3C2b618CcB071ADB122109DA96Da2
ethereum  Gelato Uniswap FRAX/DAI  0xcdfc491804A420b677f8e788B5157856910E2F6f
ethereum  StakeDAO sdETH-FraxPut   0x0A53544b2194Dd8Ebc62c779043fc0624705BB56
ethereum  Uniswap V3 FRAX/agEUR    0xf8caEd1943B15B877D7105B9906a618c154f69E8
ethereum  Vesper Orbit FRAX        0x698137C473bc1F0Ea9b85adE45Caf64ef2DF48d6
ethereum  Temple FRAX/TEMPLE       0x10460d02226d6ef7B2419aE150E6377BdbB7Ef16
arbitrum  Curve VSTFRAX-f          0x127963A74c07f72D862F2Bdc225226c3251BD117


In [6]:
# fraximalist.eth (example) 
address = '0x68e912af6176bb1639aca936eb58d71cc2558b66'

# let's gather useful pool data...
for pool in pools:
  name = pool['identifier']
  if name in staking_contracts:
    staking_contracts[name]['pool'] = pool

# How to get staked info from Frax API for an address
def get_user_stake_info(address, chain, gauge_id):
  URL = 'https://api.frax.finance/stakedata/user-stake-info'
  data = {'the_chain': chain,
          'staking_choice': gauge_id,
          'staker_address': address}
  req = httpx.post(URL, data=data)
  return req.content

for gauge in gauges:
  name = gauge['name']
  contract = staking_contracts[name]
  apy = contract['pool']['apy']
  apy_max = contract['pool']['apy_max']
  chain = contract['chain']

  content = get_user_stake_info(address, chain, name)
  info = json.loads(content)
  # pprint(info)
  total_value = info['locked_total_usd_value'] + info['unlocked_total_usd_value'] + info['lp_token_bal_usd']
  print()
  print(f"{contract['chain']:10} : {gauge['name']:24} ({apy:7.2f}/{apy_max:7.2f})% : ${total_value:12,.2f}")
  for reward in info['rewards']:
    print(f"  {reward['reward_balance']:12,.3f} {reward['token']:6} (${reward['reward_usd_value']:,.2f})")
  
# We can also look onchain
'''
ABI notes...

FraxFarm_UniV3_veFXS_FRAX_USDC
is FraxFarm_UniV3_veFXS
https://etherscan.io/address/0x3EF26504dbc8Dd7B7aa3E97Bc9f3813a9FC0B4B0#readContract
earned(address)
userStakedFrax(address)
uni_token0
uni_token1
combinedWeightOf(address)
minVeFXSForMaxBoost(address)

FraxMiddlemanGauge_FRAX_mUSD
is FraxMiddlemanGauge
https://etherscan.io/address/0x3e14f6EEDCC5Bc1d0Fc7B20B45eAE7B1F74a6AeC#readContract
name
bridge_address
destination_address_override
-> you need to find the chain yourself...
https://polygonscan.com/address/0xc425Fd9Ed3C892d849C9E1a971516da1C1B29696#readContract
FraxCrossChainFarm_FRAX_mUSD
is FraxCrossChainFarm
combinedWeightOf()
earned()
lockedLiquidityOf(address)
lockedStakesOf(address)
rewardsToken0
rewardsToken1
userRewardPerTokenPaid0(address)
userRewardPerTokenPaid1(address)
stakingToken

FraxUniV3Farm_Stable_FRAX_DAI 
is FraxUniV3Farm_Stable
earned(address)
userStakedFrax(address)
uni_token0
uni_token1
combinedWeightOf(address)
minVeFXSForMaxBoost(address)

StakingRewardsMultiGauge_FRAX_SUSHI
is StakingRewardsMultiGauge
https://etherscan.io/address/0xb4Ab0dE6581FBD3A02cF8f9f265138691c3A7d5D#readContract
earned(address)
getAllRewardTokens
'''
address = Web3.toChecksumAddress(address)
'''
try:
  user_frax = c_farm.functions.userStakedFrax(user).call()
      user_frax = Web3.fromWei(user_frax, 'ether')
      print(f'FRAX balance: ${user_frax:,.2f}')
    except:
      pass
'''
print()


ethereum   : Uniswap V3 FRAX/USDC     (   8.59/  42.95)% : $  137,118.67
        61.019 FXS    ($1,357.07)
         8.404 FRAX   ($8.54)
         8.517 USDC   ($8.53)

polygon    : mStable FRAX/mUSD        (   9.02/  27.05)% : $        0.00
         0.000 FXS    ($0.00)
         0.000 MTA    ($0.00)

ethereum   : Uniswap V3 FRAX/DAI      (  12.62/  63.08)% : $  138,984.49
        79.300 FXS    ($1,763.64)
         7.320 FRAX   ($7.44)
         7.546 DAI    ($7.55)

ethereum   : Sushi FRAX/SUSHI         (  68.70/ 343.51)% : $  100,485.33
       403.176 FXS    ($8,966.63)
         0.000 SUSHI  ($0.00)

ethereum   : StakeDAO sdFRAX3CRV-f    (   7.21/  36.06)% : $  150,254.96
        46.385 FXS    ($1,031.61)
       197.452 SDT    ($227.07)

ethereum   : Gelato Uniswap FRAX/DAI  (   6.28/  31.40)% : $  141,053.07
     1,097.320 FXS    ($24,404.39)

ethereum   : StakeDAO sdETH-FraxPut   (  11.46/  57.28)% : $  154,929.32
        66.684 FXS    ($1,483.05)
       453.225 SDT    ($521.21)

et